# Modell-Export für Node-RED

Dieses Notebook exportiert trainierte Modelle in Formate, die vom **ML Inference Node** unterstützt werden:

1. **ONNX** - Für sklearn/XGBoost Modelle
2. **TensorFlow.js** - Für Keras/TensorFlow Modelle
3. **TensorFlow SavedModel** - Alternative für TensorFlow
4. **Metadata** - JSON-Konfiguration für den Node

## Unterstützte Formate im ML Inference Node

| Format | Endung | Bibliothek |
|--------|--------|------------|
| TensorFlow.js | `model.json` | @tensorflow/tfjs-node |
| ONNX | `.onnx` | onnxruntime-node |
| TensorFlow SavedModel | `saved_model.pb` | @tensorflow/tfjs-node |

In [1]:
# Installation
# !pip install numpy pandas scikit-learn tensorflow onnx skl2onnx tf2onnx tensorflowjs joblib

In [2]:
import numpy as np
import pandas as pd
import json
import os
import shutil

# Verzeichnisse
TRAINED_DIR = '../models/trained'
EXPORT_DIR = '../models/exported'

os.makedirs(EXPORT_DIR, exist_ok=True)

print("Setup abgeschlossen!")
print(f"\nVorhandene Modelle in {TRAINED_DIR}:")
if os.path.exists(TRAINED_DIR):
    for f in os.listdir(TRAINED_DIR):
        print(f"  - {f}")
else:
    print("  Keine Modelle gefunden. Bitte zuerst die anderen Notebooks ausführen.")

Setup abgeschlossen!

Vorhandene Modelle in ../models/trained:
  - mlp_classifier_tfjs
  - scaler.joblib
  - rul_cnn_lstm.keras
  - classification_confusion_matrices.png
  - xgboost_classifier.joblib
  - classification_scaler.joblib
  - classification_metadata.json
  - cnn_classifier.keras
  - random_forest_classifier.joblib
  - rul_lstm.keras
  - rul_scaler.joblib
  - rf_feature_importance.png
  - autoencoder.keras
  - pca_anomaly.joblib
  - one_class_svm.joblib
  - classification_data_overview.png
  - isolation_forest.joblib
  - autoencoder_tfjs
  - mlp_classifier.keras
  - rul_metadata.json
  - rul_data_overview.png
  - metadata.json
  - rul_gradient_boosting.joblib
  - model_comparison_roc.png
  - rul_model_comparison.png
  - isolation_forest_evaluation.png
  - gb_feature_importance.png
  - cnn_classifier_tfjs
  - autoencoder_training.png
  - lstm_training.png
  - rul_cnn_lstm_tfjs
  - rul_lstm_tfjs


## 1. ONNX Export für sklearn Modelle

In [3]:
import joblib

try:
    from skl2onnx import convert_sklearn, to_onnx
    from skl2onnx.common.data_types import FloatTensorType
    import onnx
    HAS_ONNX = True
    print("ONNX-Export verfügbar")
except ImportError:
    HAS_ONNX = False
    print("⚠ skl2onnx nicht installiert. ONNX-Export nicht verfügbar.")
    print("  Installiere mit: pip install skl2onnx onnx")

⚠ skl2onnx nicht installiert. ONNX-Export nicht verfügbar.
  Installiere mit: pip install skl2onnx onnx


In [4]:
def export_sklearn_to_onnx(model_path, output_path, n_features, model_name="model"):
    """
    Exportiert ein sklearn Modell nach ONNX.
    
    Parameters:
    -----------
    model_path : str
        Pfad zum .joblib Modell
    output_path : str
        Ausgabepfad für .onnx
    n_features : int
        Anzahl der Input-Features
    model_name : str
        Name des Modells (für Dokumentation)
    """
    if not HAS_ONNX:
        print(f"⚠ Überspringe {model_name} (ONNX nicht verfügbar)")
        return False
    
    try:
        # Modell laden
        model = joblib.load(model_path)
        
        # Input-Typ definieren
        initial_type = [('input', FloatTensorType([None, n_features]))]
        
        # Konvertieren
        onnx_model = convert_sklearn(model, initial_types=initial_type, target_opset=12)
        
        # Speichern
        with open(output_path, 'wb') as f:
            f.write(onnx_model.SerializeToString())
        
        print(f"✓ {model_name} -> {output_path}")
        return True
        
    except Exception as e:
        print(f"✗ {model_name}: {str(e)}")
        return False

In [5]:
# Anomalie-Erkennung Modelle exportieren
print("=== Anomalie-Erkennung ===")

# Metadata laden für Feature-Anzahl
try:
    with open(f'{TRAINED_DIR}/metadata.json', 'r') as f:
        anomaly_meta = json.load(f)
    n_features_anomaly = anomaly_meta['n_features']
except FileNotFoundError:
    n_features_anomaly = 10  # Fallback

# Isolation Forest
if os.path.exists(f'{TRAINED_DIR}/isolation_forest.joblib'):
    export_sklearn_to_onnx(
        f'{TRAINED_DIR}/isolation_forest.joblib',
        f'{EXPORT_DIR}/isolation_forest.onnx',
        n_features_anomaly,
        'Isolation Forest'
    )

=== Anomalie-Erkennung ===
⚠ Überspringe Isolation Forest (ONNX nicht verfügbar)


In [6]:
# Klassifikation Modelle exportieren
print("\n=== Klassifikation ===")

try:
    with open(f'{TRAINED_DIR}/classification_metadata.json', 'r') as f:
        class_meta = json.load(f)
    n_features_class = class_meta['n_features']
except FileNotFoundError:
    n_features_class = 10

# Random Forest Classifier
if os.path.exists(f'{TRAINED_DIR}/random_forest_classifier.joblib'):
    export_sklearn_to_onnx(
        f'{TRAINED_DIR}/random_forest_classifier.joblib',
        f'{EXPORT_DIR}/random_forest_classifier.onnx',
        n_features_class,
        'Random Forest Classifier'
    )


=== Klassifikation ===
⚠ Überspringe Random Forest Classifier (ONNX nicht verfügbar)


In [7]:
# RUL Modelle exportieren
print("\n=== RUL Prediction ===")

try:
    with open(f'{TRAINED_DIR}/rul_metadata.json', 'r') as f:
        rul_meta = json.load(f)
    n_features_rul = rul_meta['n_features']
except FileNotFoundError:
    n_features_rul = 10

# Gradient Boosting RUL
if os.path.exists(f'{TRAINED_DIR}/rul_gradient_boosting.joblib'):
    export_sklearn_to_onnx(
        f'{TRAINED_DIR}/rul_gradient_boosting.joblib',
        f'{EXPORT_DIR}/rul_gradient_boosting.onnx',
        n_features_rul,
        'Gradient Boosting RUL'
    )


=== RUL Prediction ===
⚠ Überspringe Gradient Boosting RUL (ONNX nicht verfügbar)


## 2. TensorFlow.js Export für Keras Modelle

In [8]:
try:
    import tensorflow as tf
    import tensorflowjs as tfjs
    HAS_TFJS = True
    print(f"TensorFlow.js Export verfügbar (TF {tf.__version__})")
except ImportError:
    HAS_TFJS = False
    print("⚠ tensorflowjs nicht installiert.")
    print("  Installiere mit: pip install tensorflowjs")

2026-01-18 17:20:07.913068: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-18 17:20:07.920117: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-18 17:20:07.936538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768753207.962937   84395 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768753207.970464   84395 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768753207.995734   84395 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

TensorFlow.js Export verfügbar (TF 2.19.0)


/home/la/private/node-red-contrib-condition-monitoring/notebooks_venv/lib/python3.12/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [9]:
def export_keras_to_tfjs(model_path, output_dir, model_name="model"):
    """
    Exportiert ein Keras Modell nach TensorFlow.js.
    
    Parameters:
    -----------
    model_path : str
        Pfad zum .keras Modell
    output_dir : str
        Ausgabeverzeichnis für TF.js Modell
    model_name : str
        Name des Modells
    """
    if not HAS_TFJS:
        print(f"⚠ Überspringe {model_name} (TF.js nicht verfügbar)")
        return False
    
    try:
        # Modell laden
        model = tf.keras.models.load_model(model_path)
        
        # Zu TensorFlow.js konvertieren
        os.makedirs(output_dir, exist_ok=True)
        tfjs.converters.save_keras_model(model, output_dir)
        
        print(f"✓ {model_name} -> {output_dir}")
        return True
        
    except Exception as e:
        print(f"✗ {model_name}: {str(e)}")
        return False

In [10]:
# Keras Modelle exportieren
print("=== TensorFlow.js Export ===")

keras_models = [
    ('autoencoder.keras', 'autoencoder_tfjs', 'Autoencoder'),
    ('mlp_classifier.keras', 'mlp_classifier_tfjs', 'MLP Classifier'),
    ('cnn_classifier.keras', 'cnn_classifier_tfjs', '1D-CNN Classifier'),
    ('rul_lstm.keras', 'rul_lstm_tfjs', 'LSTM RUL'),
    ('rul_cnn_lstm.keras', 'rul_cnn_lstm_tfjs', 'CNN-LSTM RUL'),
]

for model_file, output_name, model_name in keras_models:
    model_path = f'{TRAINED_DIR}/{model_file}'
    if os.path.exists(model_path):
        export_keras_to_tfjs(
            model_path,
            f'{EXPORT_DIR}/{output_name}',
            model_name
        )

2026-01-18 17:20:12.386622: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


=== TensorFlow.js Export ===


failed to lookup keras version from the file,
    this is likely a weight only file
✓ Autoencoder -> ../models/exported/autoencoder_tfjs
failed to lookup keras version from the file,
    this is likely a weight only file
✓ MLP Classifier -> ../models/exported/mlp_classifier_tfjs


failed to lookup keras version from the file,
    this is likely a weight only file
✓ 1D-CNN Classifier -> ../models/exported/cnn_classifier_tfjs
failed to lookup keras version from the file,
    this is likely a weight only file
✓ LSTM RUL -> ../models/exported/rul_lstm_tfjs


failed to lookup keras version from the file,
    this is likely a weight only file
✓ CNN-LSTM RUL -> ../models/exported/rul_cnn_lstm_tfjs


## 3. Keras zu ONNX Export

In [11]:
try:
    import tf2onnx
    HAS_TF2ONNX = True
    print("tf2onnx verfügbar")
except ImportError:
    HAS_TF2ONNX = False
    print("⚠ tf2onnx nicht installiert.")
    print("  Installiere mit: pip install tf2onnx")

⚠ tf2onnx nicht installiert.
  Installiere mit: pip install tf2onnx


In [12]:
def export_keras_to_onnx(model_path, output_path, model_name="model"):
    """
    Exportiert ein Keras Modell nach ONNX.
    """
    if not HAS_TF2ONNX:
        print(f"⚠ Überspringe {model_name} (tf2onnx nicht verfügbar)")
        return False
    
    try:
        import tensorflow as tf
        
        # Modell laden
        model = tf.keras.models.load_model(model_path)
        
        # Input-Signatur erstellen
        input_signature = [tf.TensorSpec(model.input_shape, tf.float32, name='input')]
        
        # Konvertieren
        onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature, opset=13)
        
        # Speichern
        with open(output_path, 'wb') as f:
            f.write(onnx_model.SerializeToString())
        
        print(f"✓ {model_name} -> {output_path}")
        return True
        
    except Exception as e:
        print(f"✗ {model_name}: {str(e)}")
        return False

In [13]:
# Keras zu ONNX
print("=== Keras zu ONNX ===")

keras_to_onnx = [
    ('autoencoder.keras', 'autoencoder.onnx', 'Autoencoder'),
    ('mlp_classifier.keras', 'mlp_classifier.onnx', 'MLP Classifier'),
]

for model_file, output_file, model_name in keras_to_onnx:
    model_path = f'{TRAINED_DIR}/{model_file}'
    if os.path.exists(model_path):
        export_keras_to_onnx(
            model_path,
            f'{EXPORT_DIR}/{output_file}',
            model_name
        )

=== Keras zu ONNX ===
⚠ Überspringe Autoencoder (tf2onnx nicht verfügbar)
⚠ Überspringe MLP Classifier (tf2onnx nicht verfügbar)


## 4. Metadata für Node-RED erstellen

In [14]:
def create_model_metadata(model_name, model_type, input_shape, output_shape, 
                          feature_names=None, class_names=None, 
                          scaler_mean=None, scaler_scale=None,
                          description=""):
    """
    Erstellt Metadata-JSON für Node-RED ML Inference Node.
    """
    metadata = {
        "name": model_name,
        "type": model_type,
        "description": description,
        "input": {
            "shape": input_shape,
            "dtype": "float32"
        },
        "output": {
            "shape": output_shape,
            "dtype": "float32"
        },
        "preprocessing": {
            "normalize": scaler_mean is not None,
            "mean": scaler_mean,
            "scale": scaler_scale
        }
    }
    
    if feature_names:
        metadata["input"]["feature_names"] = feature_names
    
    if class_names:
        metadata["output"]["class_names"] = class_names
    
    return metadata

In [15]:
# Metadata für exportierte Modelle erstellen
print("=== Metadata erstellen ===")

# Lade gespeicherte Metadata
metadata_files = {
    'anomaly': 'metadata.json',
    'classification': 'classification_metadata.json',
    'rul': 'rul_metadata.json'
}

loaded_metadata = {}
for key, filename in metadata_files.items():
    filepath = f'{TRAINED_DIR}/{filename}'
    if os.path.exists(filepath):
        with open(filepath, 'r') as f:
            loaded_metadata[key] = json.load(f)
            print(f"  Loaded: {filename}")

=== Metadata erstellen ===
  Loaded: metadata.json
  Loaded: classification_metadata.json
  Loaded: rul_metadata.json


In [16]:
# Isolation Forest Metadata
if 'anomaly' in loaded_metadata:
    meta = loaded_metadata['anomaly']
    iso_meta = create_model_metadata(
        model_name="Isolation Forest Anomaly Detector",
        model_type="anomaly_detection",
        input_shape=[None, meta['n_features']],
        output_shape=[None, 1],
        feature_names=meta.get('feature_names'),
        scaler_mean=meta.get('scaler_mean'),
        scaler_scale=meta.get('scaler_scale'),
        description="Isolation Forest für Anomalie-Erkennung in Sensordaten"
    )
    
    with open(f'{EXPORT_DIR}/isolation_forest_metadata.json', 'w') as f:
        json.dump(iso_meta, f, indent=2)
    print("✓ isolation_forest_metadata.json")

✓ isolation_forest_metadata.json


In [17]:
# Random Forest Classifier Metadata
if 'classification' in loaded_metadata:
    meta = loaded_metadata['classification']
    rf_meta = create_model_metadata(
        model_name="Random Forest Fault Classifier",
        model_type="classification",
        input_shape=[None, meta['n_features']],
        output_shape=[None, meta['n_classes']],
        feature_names=meta.get('feature_names'),
        class_names=meta.get('class_names'),
        scaler_mean=meta.get('scaler_mean'),
        scaler_scale=meta.get('scaler_scale'),
        description="Random Forest zur Klassifikation von Maschinenzuständen"
    )
    
    with open(f'{EXPORT_DIR}/random_forest_classifier_metadata.json', 'w') as f:
        json.dump(rf_meta, f, indent=2)
    print("✓ random_forest_classifier_metadata.json")

✓ random_forest_classifier_metadata.json


In [18]:
# LSTM RUL Metadata
if 'rul' in loaded_metadata:
    meta = loaded_metadata['rul']
    lstm_meta = create_model_metadata(
        model_name="LSTM RUL Predictor",
        model_type="regression",
        input_shape=[None, meta['sequence_length'], meta['n_features']],
        output_shape=[None, 1],
        feature_names=meta.get('feature_names'),
        scaler_mean=meta.get('scaler_mean'),
        scaler_scale=meta.get('scaler_scale'),
        description="LSTM zur Vorhersage der Restlebensdauer (RUL)"
    )
    lstm_meta['sequence_length'] = meta['sequence_length']
    lstm_meta['rul_cap'] = meta['rul_cap']
    
    with open(f'{EXPORT_DIR}/rul_lstm_metadata.json', 'w') as f:
        json.dump(lstm_meta, f, indent=2)
    print("✓ rul_lstm_metadata.json")

✓ rul_lstm_metadata.json


## 5. Verwendung in Node-RED

In [19]:
# Anleitung für Node-RED generieren
readme_content = """
# Exportierte Modelle für Node-RED

Diese Modelle können mit dem **ML Inference Node** in Node-RED verwendet werden.

## Verfügbare Modelle

### 1. Anomalie-Erkennung

**Isolation Forest** (ONNX)
- Datei: `isolation_forest.onnx`
- Metadata: `isolation_forest_metadata.json`
- Input: Array von Features (normalisiert)
- Output: Anomalie-Score (-1 = Anomalie, 1 = Normal)

```javascript
// Beispiel: Input für ML Inference Node
msg.payload = [0.5, 1.2, 0.8, 3.1, ...]; // Features
```

**Autoencoder** (TensorFlow.js)
- Verzeichnis: `autoencoder_tfjs/`
- Input: Array von Features (normalisiert)
- Output: Rekonstruktion (vergleiche mit Input für Anomalie-Score)

### 2. Fehlerklassifikation

**Random Forest Classifier** (ONNX)
- Datei: `random_forest_classifier.onnx`
- Klassen: normal, unbalance, bearing, misalignment
- Output: Wahrscheinlichkeiten pro Klasse

**MLP Classifier** (TensorFlow.js)
- Verzeichnis: `mlp_classifier_tfjs/`
- Klassen: normal, unbalance, bearing, misalignment

### 3. RUL Prediction

**Gradient Boosting** (ONNX)
- Datei: `rul_gradient_boosting.onnx`
- Input: Einzelnes Feature-Array
- Output: RUL in Zyklen (0-125)

**LSTM** (TensorFlow.js)
- Verzeichnis: `rul_lstm_tfjs/`
- Input: Sequenz von 30 Zeitschritten
- Output: RUL in Zyklen

## Node-RED Konfiguration

### ML Inference Node

1. **Model Path**: Pfad zum Modell (`.onnx` oder `model.json`)
2. **Model Type**: `onnx` oder `tensorflow`
3. **Input Property**: `msg.payload`
4. **Output Property**: `msg.prediction`

### Preprocessing

Die Modelle erwarten normalisierte Eingaben. Verwende die Scaler-Parameter:

```javascript
// In einer Function Node vor ML Inference
const mean = [...]; // Aus metadata.json
const scale = [...]; // Aus metadata.json

msg.payload = msg.payload.map((val, i) => (val - mean[i]) / scale[i]);
return msg;
```

### Beispiel Flow

```
[Sensor Input] -> [Feature Extraction] -> [Normalize] -> [ML Inference] -> [Postprocess] -> [Output]
```

## Dateien

| Datei | Format | Verwendung |
|-------|--------|------------|
| `*.onnx` | ONNX | ML Inference mit onnxruntime |
| `*_tfjs/model.json` | TensorFlow.js | ML Inference mit @tensorflow/tfjs |
| `*_metadata.json` | JSON | Preprocessing & Konfiguration |
"""

with open(f'{EXPORT_DIR}/README.md', 'w') as f:
    f.write(readme_content)

print("✓ README.md erstellt")

✓ README.md erstellt


## 6. Export-Zusammenfassung

In [20]:
# Übersicht der exportierten Dateien
print("=" * 60)
print("EXPORT ZUSAMMENFASSUNG")
print("=" * 60)
print(f"\nExport-Verzeichnis: {os.path.abspath(EXPORT_DIR)}")
print("\nExportierte Dateien:")

total_size = 0
for root, dirs, files in os.walk(EXPORT_DIR):
    level = root.replace(EXPORT_DIR, '').count(os.sep)
    indent = '  ' * level
    
    if root == EXPORT_DIR:
        for f in files:
            filepath = os.path.join(root, f)
            size = os.path.getsize(filepath) / 1024  # KB
            total_size += size
            print(f"  {f} ({size:.1f} KB)")
    
    for d in dirs:
        dirpath = os.path.join(root, d)
        dir_size = sum(os.path.getsize(os.path.join(dirpath, f)) for f in os.listdir(dirpath) if os.path.isfile(os.path.join(dirpath, f))) / 1024
        total_size += dir_size
        print(f"  {d}/ ({dir_size:.1f} KB)")

print(f"\nGesamtgröße: {total_size/1024:.2f} MB")

EXPORT ZUSAMMENFASSUNG

Export-Verzeichnis: /home/la/private/node-red-contrib-condition-monitoring/models/exported

Exportierte Dateien:
  rul_lstm_metadata.json (1.5 KB)
  README.md (2.2 KB)
  isolation_forest_metadata.json (1.5 KB)
  random_forest_classifier_metadata.json (1.5 KB)
  mlp_classifier_tfjs/ (22.5 KB)
  autoencoder_tfjs/ (22.4 KB)
  cnn_classifier_tfjs/ (184.5 KB)
  rul_cnn_lstm_tfjs/ (78.7 KB)
  rul_lstm_tfjs/ (141.3 KB)

Gesamtgröße: 0.45 MB


In [21]:
# Validierung: ONNX Modelle testen
if HAS_ONNX:
    import onnx
    
    print("\n=== ONNX Validierung ===")
    
    for f in os.listdir(EXPORT_DIR):
        if f.endswith('.onnx'):
            try:
                model = onnx.load(f'{EXPORT_DIR}/{f}')
                onnx.checker.check_model(model)
                print(f"✓ {f} - Valid")
            except Exception as e:
                print(f"✗ {f} - {str(e)}")

## Zusammenfassung

### Exportierte Formate

| Modell | ONNX | TensorFlow.js | Empfehlung |
|--------|------|---------------|------------|
| Isolation Forest | ✓ | - | ONNX |
| Random Forest | ✓ | - | ONNX |
| Gradient Boosting | ✓ | - | ONNX |
| Autoencoder | ✓ | ✓ | TF.js |
| MLP | ✓ | ✓ | TF.js |
| LSTM | - | ✓ | TF.js |
| CNN-LSTM | - | ✓ | TF.js |

### Nächste Schritte

1. Kopiere die exportierten Modelle in das Node-RED Verzeichnis
2. Konfiguriere den ML Inference Node mit dem Modellpfad
3. Implementiere Preprocessing (Normalisierung) falls nötig
4. Teste mit echten Sensordaten